__When having a machine learning problem you have to select the right model and right parameter values for your model. How do you do that?__

In [5]:
# choosing to work with SVM
from sklearn import svm, datasets
iris = datasets.load_iris()
dir(iris)

['DESCR', 'data', 'feature_names', 'filename', 'target', 'target_names']

In [6]:
iris.feature_names #this feature_names will be our column names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [8]:
import pandas as pd
df = pd.DataFrame(iris.data, columns = iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


In [9]:
# the tradition approach to solve the problem of "best machine learning model" is using train_test_split method
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(iris.data,iris.target,test_size=0.3)

In [10]:
# training and getting the score
model = svm.SVC(kernel='rbf', C=10, gamma='auto')
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.9555555555555556

In [11]:
# I can not rely on the method above because my scores keeps on changing whenever I rerun train_test_split
# so we use K-fold cross validation
from sklearn.model_selection import cross_val_score
# down here we tune the parameters and see what happens to the score

In [16]:
cross_val_score(svm.SVC(kernel='rbf', C=10, gamma='auto'), iris.data,iris.target, cv=5) # cv specifies number of folds

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [17]:
cross_val_score(svm.SVC(kernel='linear', C=10, gamma='auto'), iris.data,iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [18]:
cross_val_score(svm.SVC(kernel='rbf', C=20, gamma='auto'), iris.data,iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [23]:
# tuning the parameters on top here is manual and repetitive, let's use a for loop
import numpy as np
kernels = ['rbf', 'linear']
c = [1,10,20]
avg_scores = {}
for kernelVal in kernels:
    for cval in c:
        cv_scores = cross_val_score(svm.SVC(kernel=kernelVal, C=cval, gamma='auto'), iris.data,iris.target, cv=5)
        avg_scores[kernelVal+'_'+str(cval)] = np.average(cv_scores)
avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

__GridSearchCV for hyperparameter tuning__

In [24]:
# but the for loop has its throw backs when you have a huge number of parameters; you have to run a lot of 
# for loops which is inconvinient. luckily sklearn provides GridSearchCV which will do the same thing conviniently
from sklearn.model_selection import GridSearchCV 

clf = GridSearchCV(svm.SVC(gamma='auto'),         # keeping gamma value static
                   {# you supplly all your parameters in this dict
                       'C':[1,10,20],
                       'kernel':['rbf','linear']
                   },
                   cv = 5,                       # specifying the number of cross validations/folds
                   return_train_score=False
                  )
# doing the model training
clf.fit(iris.data,iris.target)
clf.cv_results_

{'mean_fit_time': array([0.0010004 , 0.0005991 , 0.00099988, 0.00079966, 0.00119944,
        0.00120101]),
 'std_fit_time': array([6.32037697e-04, 4.89161660e-04, 1.56268784e-06, 3.99828313e-04,
        3.98184050e-04, 7.50479605e-04]),
 'mean_score_time': array([0.00079966, 0.00040016, 0.00019875, 0.00039964, 0.00119901,
        0.00079737]),
 'std_score_time': array([0.00039983, 0.0004901 , 0.00039749, 0.00048945, 0.00146913,
        0.00039871]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20

In [25]:
# downloading the cv_results_ into a dataframe
cv_results_df = pd.DataFrame(clf.cv_results_)
cv_results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001000,0.000632,0.000800,0.000400,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000599,0.000489,0.000400,0.000490,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001000,0.000002,0.000199,0.000397,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000800,0.000400,0.000400,0.000489,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001199,0.000398,0.001199,0.001469,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.001201,0.000750,0.000797,0.000399,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [27]:
# trimming the unusefull columns 
cv_results_df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [28]:
# what other properties does GridSearchCV has
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_is_fitted',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_required_parameters',
 '_run_search',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'iid',
 'inverse_transform',
 'multimetric_',
 'n_jobs',
 'n_splits_',
 'param_grid',
 'pre_dispatch',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'refit',
 'refit_time_',
 'return_train_score',
 'score

In [29]:
# getting the best score 
clf.best_score_

0.9800000000000001

In [30]:
# getting the best parameters values to use
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

__RandomizedSearchCV for hyperparameter tuning__

In [31]:
# in GridSearchCV there is high computational power when the dataset is huge and to tackle this we use RandomizedSearchCV
# this will not try every single permutation and combination of parameters but it will do it in random manner.

In [34]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'),
                      {
                          'C':[1,10,20],
                          'kernel':['rbf','linear']
                      },
                      cv=5,
                      return_train_score=False,
                      n_iter=2   # doing only 2 combinations on GridSearchCV it did 6 
                     )
rs.fit(iris.data,iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]
# from the output it randomly two of the combinations; this is effective if computer computation power is lower

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,20,linear,0.966667


### How to choose the best machine learning model

In [35]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [53]:
# defining the parameter grid
model_params = {
    'svm':{
        'model': SVC(gamma='auto'),
        'params': {
            'C':[1,10,20],
            'kernel': ['rbf','linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression':{
        'model': LogisticRegression(max_iter=4700),
        'params': {
            'C':[1,5,10]
        }
    }
}

In [54]:
# using a for loop to automate the whole process
scores = []

for model_name, mp in model_params.items(): # model_name is the key in the dictionery and mp is the value(contains model and params)
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

In [56]:
# coverting the scores results into a pandas dataframe
results_df = pd.DataFrame(scores)
results_df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 5}
2,logistic_regression,0.973333,{'C': 1}
